In [ ]:
!pip install tsplib95

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.8 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.21.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.


In [ ]:
# Run 1: population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 558391.3561074291
Generation 2: Best Cost = 558391.3561074291
Generation 3: Best Cost = 558391.3561074291
Generation 4: Best Cost = 558391.3561074291
Generation 5: Best Cost = 558391.3561074291
Generation 6: Best Cost = 558391.3561074291
Best solution found: [ 77 115  20 190  13  29  58 307 291 254  99 137  28 290 296  35 304 204
 276 280 201 105 273 258  47  55 216 205  25 260 110  44 277 272 212 275
   2 161 214  90 225  15 262 249 142 130  23 172 112  26 185   5  86 170
 173 128 155 244 168  18 227  51 135 162 198 237  14  22  21  40  92 182
  98  49  52 138 169 109 154  57 226 197 255 207 176  54  95 312 228  17
 150  34  79 106 289 160 274 235 279 174 247 261 266  56 221  48 215 179
 243 191 267 113 111 236 217 285  72 104 264 316  76  50 192  27 220 283
 157 178 180 117 308 294 245 300 251 229 140 153  85  68   6 141  19  63
  89 119 232 265 282  84 301 263 299 206 181  38 211 124  53 158 233  97
 171  60   4  74 122 238 118 209 203 152  43  12 167 293 1

In [ ]:
# Run 2: population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 543177.5466126503
Generation 2: Best Cost = 543177.5466126503
Generation 3: Best Cost = 543177.5466126503
Generation 4: Best Cost = 543177.5466126503
Generation 5: Best Cost = 543177.5466126503
Generation 6: Best Cost = 543177.5466126503
Best solution found: [181  44 254  20 274  16 190 100  68 179 209 261 215  22 123 113 256  53
 292 129 288 219 140  33  89  40  87 175 234 297 176 271 112 248 255 223
 295  27  49 146  77  15 154   1  65 206 201 277  42  67 303 299  74 132
 216 170 226 212 193  76  78  10 117 231 312 120 293 214 160 148 142 310
 166 167 109 269 265  70 169 202 273 119 244  92 300 199 195 198 135   7
  25 189 103 301  99 230 263 247  96 173 185   2  58  54 203  17 242 249
 257 314 153  38  36 183 134 172 104  13 232 152  94 224   5 186 204  23
 246 192  32 281  63 110  46 306 289  59  60 163  80   6 305  37  50 115
  56 298 210 126 284  69 182 180 177  61 282 191 290 168 187  85 278 131
 125 241 151 275  57 225 240 307 252   8 111  43 159 266  

In [ ]:
# Run 3: population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 546611.5264886856
Generation 2: Best Cost = 546611.5264886856
Generation 3: Best Cost = 546611.5264886856
Generation 4: Best Cost = 546611.5264886856
Generation 5: Best Cost = 546611.5264886856
Generation 6: Best Cost = 546611.5264886856
Best solution found: [ 57 180 193 211  59 218 231  31  24 102 133 247  27 268 110 141  16  55
 246 290 291 300 115 285 109 272 113 303  58 122 120  29 315  73 183 316
  69 295 309 220 126  26 107  62  60 172   7 181  15   4 264 214 163 130
 296 252 131 136 186 212 238 182  41 170 152 201  30  96  67 165 313 125
 106 223 230 171 197 271   0   5 229 317 288 227 276 281 137 278 280 225
  93  14  88  68  39 194 132 196 207  32   8 273 112  78 138 310 174 144
 240 148 255  44  21 258 233 199  82  38 224  47  11 253 293 312 242 228
 219  17 166 274  23  19 116 243  22 156  74  51  97 168  89 140  75  64
 235 111 259 184 265 314 308 304 249 297 151 150 279 306 222  35 257 118
 277 244 191 245  20  34 139 147  84  36 177  52 237 251  

In [ ]:
# Run 4: population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 546857.4860855648
Generation 2: Best Cost = 546857.4860855648
Generation 3: Best Cost = 546857.4860855648
Generation 4: Best Cost = 546857.4860855648
Generation 5: Best Cost = 546857.4860855648
Generation 6: Best Cost = 546857.4860855648
Best solution found: [121 221 217 163 162  96  79  40 230 274   5 302 108 222 305 232 281 107
 127 240 225 138 180  98 194 315 250  14  28 114  24  33 137  19   0  52
  93 212 113 215  21  25 185  76 209 199  95 102  81 195 248 244 255   8
 133 251 130 223  60 170 179 213 211  42 260 135 312 152 140 144 242 239
  37 124  39 258  43 200 158  74 241 224  10  99 161 275 119 105 165  62
  22   4 227 292 136  70 148 167 210  67 159 129  65 141 166 172 300 284
 267  89  30   6 295 164  72 306 279 262 157 298 201 297 263 145 290 220
  82 178  31 177 143 301 169 283 216 207 257 264  16  73  61 206  29  49
 272 245   2 198  55 139 193 246  47 234 214  44  50 286 233 228 311 192
 142 132  64  86 307  32  17 176 110  68 112 226 219 175 2

In [ ]:
# Run 5: population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 556981.037861986
Generation 2: Best Cost = 556981.037861986
Generation 3: Best Cost = 556981.037861986
Generation 4: Best Cost = 556981.037861986
Generation 5: Best Cost = 556981.037861986
Generation 6: Best Cost = 556981.037861986
Best solution found: [302  72 275 159  47 297  21 160 216 226 283 287 253  76 104 209 230 217
  65 189 311 306 250 276 184  20 260 227 221 239 177  62 153 149 194  71
 202 188 266 312  18  22 126 286 258   9  51 157 229 142 207 304 110 136
  82 251 211 180 203 172 109  38  66  86 121 125  64 315 129  68  63  14
  13  43 111  78  45  83 205 171 166 196 122 222  77 281 249  24 236 138
 268  31 150 282 295 162 127 195 199 108 313   7 314 100  94  98 206 152
 262 137 145  28 151 293 213 233  29 237 247 214 243   4 154  32 228 133
 158  15   3 204  69  17 168 305  61 131  54 290 135  40 245 178 252 254
 155  27  41 165 120 255 114   1  96  88 248  26   6 317 117   8 190  53
  33  90 294 124 269 119 115  36 146 107  93 118 273 140 263  57

In [ ]:
# Run 6: population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 543281.7692123719
Generation 2: Best Cost = 543281.7692123719
Generation 3: Best Cost = 543281.7692123719
Generation 4: Best Cost = 543281.7692123719
Generation 5: Best Cost = 543281.7692123719
Generation 6: Best Cost = 543281.7692123719
Best solution found: [  4 194 207 317 219 285 182 199 206 287  83 309  13 275 112 235 240 222
  96  80 198 274 148 295 256   0  34  35 163 158  70  64 154 234  57 299
  87 269   9 160 139 100 126  46 111  60 116 142 101 305 189 172 183 314
  40  54 264 152 281 270 259 115  42 119 170  51 279 106 186 167 129 145
 188 298 252 245 216 308 195  98  59 181 302  50 177  85  10 133 136 123
 175 117 103  82   6  19  12 248 277 300 214  66 265 273 138  39  49 184
 187 156 313 251 282 204 257 137  93 226 290  77 173 244  92  90  30 179
  26 215  24 241 247 310 192 164 211  73  67 120  86 147  20 109 297   5
 131  33 151 231 306 303 280  22  29 157 165 230 130 155  58 144 153  11
 224 134  16 271 312   3  23 196 286 218  68 272  94  52 2

In [ ]:
# Run 7: population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 546610.0722576515
Generation 2: Best Cost = 546610.0722576515
Generation 3: Best Cost = 546610.0722576515
Generation 4: Best Cost = 546610.0722576515
Generation 5: Best Cost = 546610.0722576515
Generation 6: Best Cost = 546610.0722576515
Best solution found: [151 267  46 240 317  43  11 256 152   7 159 258 238 299 138 294 295 163
  89  91  52  79 171 287 290 268  16  19  62  76 175 308 176 160 237 170
 316 194 137 273 115 310  80 131  54  36  74 177 150 182 250  15 314 252
 227 282 118 169 162 148 247  12 300 121  83 225 166 234 279 288  85  55
  86 189 245 205 204 158  67 217 196 249 206 161 198 101  33 289 307 277
 154 266  44  10 122  81 210 244 213 296 156  98  25  20 229 272 188  73
 257  26 220  75 153 203 255 109   9  53 140 269 124  39 103  40 193  94
 133 129 270 191 304 246 195  99  30  21 241   8  41 173   1 178 212 313
  61 100  29  68  49  50 283 298 197 218 128 315  34  90 113 112  72  45
  47 265 116 202  65 216 199 232 157  48 135 174 239 132 1

In [ ]:
# Run 8: population_size=300, max_gen=6, elite_count=20, mutation_prob=1
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 554765.3108661576
Generation 2: Best Cost = 554765.3108661576
Generation 3: Best Cost = 554765.3108661576
Generation 4: Best Cost = 554765.3108661576
Generation 5: Best Cost = 554765.3108661576
Generation 6: Best Cost = 554765.3108661576
Best solution found: [142  83 111 308 211 314  12  93 246  86 295   8  79 301  96 231 239  30
 112 153 250 109 212  61 294  60 269 313  37 273  74 175 122 204  50  76
  35 315   6 266 264 293 203   0  89 207 177 116  16  24  78  39   7 312
 234 180  11 196 280   9 149  58  71 265 260 202 184  32  46  42 178 197
  84 227 224 253 118  44 146 286 163 291 160 305 100  67  65 174 217 132
  92 124  19 258 139 117 172 233 285 281 311 229 158 148 284 120 236 263
  29 310 105 303 107 226 189 219 176 185 309 133  48  70 270 162   5  80
 152 259  40  33 288  34  98 287 262 245 221 249 261  53 123 110 113 200
  31 208 317 216 147 289 276 277 182 238 214 127  13  62  68 136  47 179
 290 190 193 154 144 138 131  91 296 143 254 283  55 170 1

In [ ]:
# Run 9: population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 546893.5970645202
Generation 2: Best Cost = 546893.5970645202
Generation 3: Best Cost = 546893.5970645202
Generation 4: Best Cost = 546893.5970645202
Generation 5: Best Cost = 546893.5970645202
Generation 6: Best Cost = 546893.5970645202
Best solution found: [ 50 216 227 187 272 203 115 133 125 287  93  60 110 132 299 270  57  67
   9  76 289 251  29  87 160 145 303 182  89 217  85 165 109 117  20 101
 143 214  64 200  95  51 257 258   1 222 172  13 207 202 176 169 291 266
 293 260 122 225 261 254  45 158 138  68 271  12  35  22 205 195 253 256
  21 310 126 305 273 130 171  49  30  79 153 166 263 294 157 309  58 292
 106 148 104 137  91  41  40 156 243  16  75  54 179 201 268   7  39  59
  71  69 265 314 111  70 277 142 184 278 128 307 264 196 224 231  98   3
 248 181  84 161 163 154 213 296  63 226  32 113   4 247 223 241  28  33
 302 211 297 167 250 295 189  26 124 121 116 135 174  19 102  14  10 281
 252 183 308 242 312 249 162 238 269 244 240 210 151 275 2

In [ ]:
# Run 10: population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/lin318.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 550402.5979274563
Generation 2: Best Cost = 550402.5979274563
Generation 3: Best Cost = 550402.5979274563
Generation 4: Best Cost = 550402.5979274563
Generation 5: Best Cost = 550402.5979274563
Generation 6: Best Cost = 550402.5979274563
Best solution found: [ 95  50  69   2 203 231  22  73 186 134 110 199 119 106  19  44 261 205
 291 239  31 145   9 165 254  76  81 255  96 249 228  45 111  24  98  60
 166  59 144  97 225 271  43 267   0  80 168 270  57 232 184 262 266 312
  35 171 169 115 285 290 195 125 315  62  51   5 260 148 277 130  52 246
 297  55  63  14 137  41  27  86 278 174  78 206 129 210 198  10  29  87
  93  68  18  83 124 107 209  89 103 214 223  13   6 240 236 212 235  34
 188  65   3  32  26 147  39 151 162 141 153 118 143 253 175 185   1 250
  25 167  47 305 173 150 120   4 288 178 289 263 133 140  91  79 264 298
 300 294 279 292  74 123 211 164 280 101 309  92 202 241 276 136 215 227
 191  46   8 308 158  94 218 234 311 161 172 220 299 194  